## End-to-end Agent

In [1]:
TAVILY_API_KEY = ""

from langchain_ollama import ChatOllama

model = ChatOllama(
    model = "qwen3",
    temperture = 0
)

In [2]:
# Import relevant functionality
from langchain_tavily import TavilySearch
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

# Create the agent
memory = MemorySaver()
search = TavilySearch(max_results=2, tavily_api_key = TAVILY_API_KEY)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

In [3]:
# Use the agent
config = {"configurable": {"thread_id": "abc123"}}

input_message = {
    "role": "user",
    "content": "Hi, I'm Majid and I live in Algeria.",
}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Hi, I'm Majid and I live in Algeria.
================================== Ai Message ==================================

<think>
Okay, the user introduced himself as Majid living in Algeria. Let me think about how to respond.

First, I should greet him and acknowledge his introduction. Maybe say something like "Hello Majid!" to be friendly. Then, since he mentioned Algeria, I can ask how he's doing or if there's anything specific he needs help with. Algeria is a country in North Africa, so maybe he's looking for information related to his location. I should keep the tone welcoming and open-ended to encourage him to share more. Let me check if there are any tools I need to use here. The tools provided are for searching, but since he just introduced himself, there's no immediate need for a function call. Just a polite response should suffice. Alright, time to draft a response that's helpful and engaging.
</th

In [4]:
input_message = {
    "role": "user",
    "content": "What's the weather where I live?",
}

for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What's the weather where I live?
================================== Ai Message ==================================

<think>
Okay, the user asked, "What's the weather where I live?" He mentioned he's in Algeria. I need to find the current weather in Algeria. Let me check the tools available. The only tool provided is tavily_search, which is for general searches. Since there's no specific weather tool, I should use tavily_search to look up the current weather in Algeria. I'll set the query to "current weather in Algeria" and use the advanced search depth. No date ranges are needed unless specified, so I'll leave those parameters out. Make sure to include the location in the search query to get accurate results.
</think>
Tool Calls:
  tavily_search (8ea3d803-4b59-4832-b1da-f9bd51cffb20)
 Call ID: 8ea3d803-4b59-4832-b1da-f9bd51cffb20
  Args:
    query: current weather in Algeria
    search_depth: advanced
====

## Define tools

In [6]:
from langchain_tavily import TavilySearch

search = TavilySearch(max_results=2, tavily_api_key = TAVILY_API_KEY)
search_results = search.invoke("What is the weather in Tlemcen")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

{'query': 'What is the weather in Tlemcen', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Weather in Tlemcen', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Tlemcen', 'region': 'Tlemcen', 'country': 'Algeria', 'lat': 34.8783, 'lon': -1.315, 'tz_id': 'Africa/Algiers', 'localtime_epoch': 1754403246, 'localtime': '2025-08-05 15:14'}, 'current': {'last_updated_epoch': 1754402400, 'last_updated': '2025-08-05 15:00', 'temp_c': 34.0, 'temp_f': 93.2, 'is_day': 1, 'condition': {'text': 'Sunny', 'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png', 'code': 1000}, 'wind_mph': 12.5, 'wind_kph': 20.2, 'wind_degree': 347, 'wind_dir': 'NNW', 'pressure_mb': 1018.0, 'pressure_in': 30.06, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 41, 'cloud': 0, 'feelslike_c': 32.4, 'feelslike_f': 90.3, 'windchill_c': 36.0, 'windchill_f': 96.7, 'heatindex_c': 34.9, 'heatindex_f': 94.8, 'dewpoint_c': 11.1, 'dewpoint_f': 51.9, 'vis_km': 10.0, 'vis_mi

In [7]:
from langchain_ollama import ChatOllama

model = ChatOllama(
    model = "qwen3",
    temperture = 0
)

In [8]:
query = "Hi!"
response = model.invoke([{"role": "user", "content": query}])
response.text()

'<think>\nOkay, the user said "Hi!" so they\'re probably just greeting me. I should respond in a friendly and welcoming way. Let me make sure to keep the tone positive and open for conversation. Maybe add an emoji to keep it casual. I should ask how I can assist them today. Let me check for any typos or errors. Yep, that looks good. Ready to respond.\n</think>\n\nHello! 😊 How can I assist you today?'

In [10]:
# to give the language model knowledge of tools
model_with_tools = model.bind_tools(tools)

In [11]:
query = "Hi!"
response = model_with_tools.invoke([{"role": "user", "content": query}])

print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}")

Message content: <think>
Okay, the user sent "Hi!" so they're probably just greeting me. I need to respond politely. Let me check if there's any function I need to call here. The tools provided are for searching, but since the user didn't ask a question or request information, there's no need to use the tavily_search function. I should just reply with a friendly greeting and offer assistance. No tool calls required here.
</think>

Hello! How can I assist you today? 😊

Tool calls: []


In [12]:
query = "Search for the weather in Tlemcen"
response = model_with_tools.invoke([{"role": "user", "content": query}])

print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}")

Message content: <think>
Okay, the user wants to search for the weather in Tlemcen. Let me think about how to approach this. The available tool is tavily_search, which is a search engine optimized for accurate and trusted results. But wait, the function parameters include things like query, end_date, exclude_domains, etc. The user's query is straightforward: weather in Tlemcen.

First, I need to determine if the tavily_search function is the right tool here. Since the user is asking for current weather information, which is typically real-time data, the search function might not be the best choice. However, if the tool can retrieve current weather data through a search, then it's possible. But looking at the parameters, there's no specific parameter for weather or location-based searches. The function is more for general web searches, possibly including weather websites or services.

Wait, the function's description mentions it's optimized for comprehensive, accurate, and trusted resul

## Create an Agent

In [13]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

## Run the Agent

In [14]:
input_message = {"role": "user", "content": "Hi!"}
response = agent_executor.invoke({"messages": [input_message]})

for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

Hi!
================================== Ai Message ==================================

<think>
Okay, the user said "Hi!" so they're probably just greeting me. I need to respond politely. Since there's no specific question or request, I don't need to use any tools here. I'll just reply with a friendly message to start the conversation.
</think>

Hello! How can I assist you today? 😊


In [15]:
input_message = {"role": "user", "content": "Search for the weather in Tlemcen"}
response = agent_executor.invoke({"messages": [input_message]})

for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

Search for the weather in Tlemcen
================================== Ai Message ==================================

<think>
Okay, the user wants to search for the weather in Tlemcen. Let me think about how to approach this. The available tool is tavily_search, which is a search engine optimized for accurate and trusted results. But wait, the function parameters include things like query, end_date, exclude_domains, etc. The main parameter needed here is the query. The user's request is straightforward: weather in Tlemcen. Since the tool is for general searches, I should use the query parameter with "weather in Tlemcen". I don't need any additional parameters like dates or domains unless specified. The user didn't mention any specific date range or exclude certain sites, so the default settings should work. Let me make sure the function is called correctly. The name of the function is tavily_search, and the

## Streaming Messages

In [16]:
for step in agent_executor.stream({"messages": [input_message]}, stream_mode="values"):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Search for the weather in Tlemcen
================================== Ai Message ==================================

<think>
Okay, the user wants to search for the weather in Tlemcen. Let me think about how to approach this. The available tool is the tavily_search function, which is optimized for comprehensive and accurate results. But wait, the function is described as a search engine for general information, possibly including news and current events. However, weather information might not be directly covered by this tool unless it's part of recent articles or data.

First, I need to check if the tavily_search can retrieve weather data. The parameters include query, domains, dates, etc. Since there's no specific weather parameter, I might need to use the query field to ask for weather information. But the tool's description mentions it's optimized for current events and accurate results, so maybe it can 

In [17]:
for step, metadata in agent_executor.stream(
    {"messages": [input_message]}, stream_mode="messages"
):
    if metadata["langgraph_node"] == "agent" and (text := step.text()):
        print(text, end="|")

<think>|
|Okay|,| the| user| wants| to| search| for| the| weather| in| T|lem|cen|.| Let| me| think| about| how| to| approach| this|.| The| available| tool| is| the| tav|ily|_search| function|,| which| is| for| comprehensive| and| accurate| results|.| But| wait|,| the| function|'s| description| mentions| it|'s| optimized| for| current| events| and| trusted| results|.| However|,| weather| information| might| not| be| the| strongest| use| case| for| this| tool|.| Typically|,| weather| data| is| more| accurately| retrieved| from| specialized| weather| services| or| APIs|.| But| since| the| user| hasn|'t| provided| access| to| a| specific| weather| API|,| maybe| I| should| still| use| the| search| function| to| find| the| latest| weather| information|.

|First|,| I| need| to| check| if| the| tav|ily|_search| function| can| handle| this|.| The| parameters| include| query|,| start|_date|,| end|_date|,| and| others|.| The| query| parameter| is| required|,| so| I|'ll| set| that| to| "|weather| 

## Adding in memory

In [22]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [23]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [24]:
input_message = {"role": "user", "content": "Hi, I'm Majid!"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Hi, I'm Majid!
================================== Ai Message ==================================

<think>
Okay, the user introduced themselves as Majid. I should respond warmly and ask how I can assist them. Since there's no specific query yet, I'll keep it open-ended and friendly. No need to call any functions here because the conversation is just starting. Let me make sure to welcome them and invite them to ask for help with anything they need.
</think>

Hello, Majid! Welcome! How can I assist you today? If you have any questions or need help with something, feel free to ask! 😊


In [25]:
input_message = {"role": "user", "content": "What's my name?"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What's my name?
================================== Ai Message ==================================

<think>
Okay, the user asked, "What's my name?" Let me check the conversation history.

Looking back, the user introduced themselves as Majid in the first message. The assistant responded warmly and asked how they could help. Now, the user is asking for their name again. 

Hmm, maybe the user is testing if I remember previous interactions. Since the name was provided earlier, the correct answer is Majid. I should confirm that and offer further assistance. No need to use any tools here because it's a straightforward recall of the user's name from the history. Just a friendly response to reaffirm their name and ask how I can help.
</think>

Your name is Majid! 😊 How can I assist you today?


In [26]:
config = {"configurable": {"thread_id": "xyz123"}}

input_message = {"role": "user", "content": "What's my name?"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What's my name?
================================== Ai Message ==================================

<think>
Okay, the user is asking, "What's my name?" Hmm, I need to figure out how to respond. Let me check the tools provided to see if any can help. The only tool available is tavily_search, which is for searching the web. But the user is asking for their own name, which isn't something that can be found through a web search. The assistant doesn't have access to personal information about the user. So, I should inform the user that I can't retrieve their name and suggest they provide it if they want to proceed with the conversation.
</think>

I don't have access to your personal information or account details. If you'd like to proceed with the conversation, please let me know your name!
